# TensorFlow

*TensorFlow* is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale Machine Learning. Its basic principle is simple: you first define in Python a graph of computations to perform, and then TensorFlow takes that graph and runs it efficiently using optimized C++ code. Most importantly, it is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs.

In [3]:
import tensorflow as tf

C:\Users\hheim\Anaconda3\envs\TensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
!conda list

# packages in environment at C:\Users\hheim\Anaconda3\envs\TensorFlow:
#
# Name                    Version                   Build  Channel
_tflow_select             2.1.0                       gpu    anaconda
absl-py                   0.9.0                    py36_0  
argon2-cffi               20.1.0                   pypi_0    pypi
astor                     0.8.1                    py36_0  
attrs                     19.3.0                   pypi_0    pypi
backcall                  0.2.0                      py_0  
blas                      1.0                         mkl  
bleach                    3.1.5                    pypi_0    pypi
ca-certificates           2020.6.24                     0  
certifi                   2020.6.20                py36_0  
cffi                      1.14.2                   pypi_0    pypi
colorama                  0.4.3                      py_0  
cudatoolkit               9.0                           1    anaconda
decorator                 4.4.2     

h5py                      2.7.1            py36he54a1c3_0    anaconda
hdf5                      1.10.1           vc14hb361328_0  [vc14]  anaconda
icc_rt                    2019.0.0             h0cc432a_1    anaconda
importlib-metadata        1.7.0                    py36_0  
intel-openmp              2020.1                      216  
ipykernel                 5.3.4            py36h5ca1d4c_0  
ipython                   7.16.1           py36h5ca1d4c_0  
ipython_genutils          0.2.0                    py36_0  
ipywidgets                7.5.1                    pypi_0    pypi
jedi                      0.17.2                   py36_0  
jinja2                    2.11.2                   pypi_0    pypi
jsonschema                3.2.0                    pypi_0    pypi
jupyter                   1.0.0                    pypi_0    pypi
jupyter-console           6.1.0                    pypi_0    pypi
jupyter_client            6.1.6                      py_0  
jupyter_core              4.6.3   

In [7]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

The most important thing to understand is that this code does not actually perform any computation, even though it looks like it does (especially the last line). It just creates a computation graph. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate.

A variable starts its life when its initializer is run, and it ends when the session is closed.

In [13]:
with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    res = sess.run(f)
    print(res)
    
tf.reset_default_graph()

42


Any instance of a node that is created is added to the default graph. If you want separate graphs do the following:

**TIP**

In Jupyter (or in a Python shell), it is common to run the same commands more than once while you are experimenting. As a
result, you may end up with a default graph containing many duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient solution is to just reset the default graph by running tf.reset_default_graph().

In [37]:
graph_1 = tf.Graph()

with graph_1.as_default():
    x1 = tf.Variable(1)

## Linear Regression with TensorFlow

In the Python API tensors are simply represented by NumPy ndarrays, you can also use `transpose()`, `matmul()`, and `matrix_inverse()`.

In [83]:
import numpy as np

np.random.seed(1234)

X=np.random.rand(1000,3)
B=np.array([1,2,3]).reshape(3,1)
Y=X@B+np.random.randn(1000,1)*5
B_hat = np.linalg.inv(X.transpose()@X)@X.transpose()@Y


Xt = tf.constant(X, dtype=tf.float32, name='X') #Source operation
Yt = tf.constant(Y, dtype=tf.float32, name='Y') #Source operation
Xt_T = tf.transpose(Xt)
Betas = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(Xt_T,Xt)),Xt_T),Yt)

with tf.Session() as LR_sess:
    Betas_LR = Betas.eval()
    Betas_LR = Betas_LR.reshape(3,1)
    print("numpy: ")
    print(B_hat)
    print('tensorflow: ')
    print(Betas_LR)

numpy: 
[[1.36967319]
 [1.61575796]
 [3.21015672]]
tensorflow: 
[[1.3696747]
 [1.615756 ]
 [3.210157 ]]


## Batch Gradient Descent

1) The `random_uniform()` function creates a node in the graph that will generate a tensor containing random values, given its shape and value range, much like NumPy’s `rand()` function.

2) The `assign()` function creates a node that will assign a new value to a variable. In this case, it implements the Batch Gradient Descent step θ(next step) = θ – η∇θMSE(θ).

3) The main loop executes the training step over and over again (n_epochs times), and every 100 iterations it prints out the current Mean Squared Error (mse). You should see the MSE go down at every iteration.

In [88]:
n_epochs = 1000
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([3, 1], minval=-5.0, maxval=5.0), name="theta") #Initialize random theta
y_pred = tf.matmul(Xt, theta, name="predictions") #Predictions to calculate MSE
error = y_pred - Yt
mse = tf.reduce_mean(tf.square(error), name="mse") #MSE
gradients = 2/1000 * tf.matmul(Xt_T, error) #Gradient Descent (based on LR Formula)
#gradients = tf.gradients(mse, [theta])[0] --> alternatively use this for automatic partial derivatives (mse/theta)
training_op = tf.assign(theta, theta - learning_rate * gradients) #Assign new value to theta

init = tf.global_variables_initializer() #Initialize all variables when called

with tf.Session() as sess:
    sess.run(init) #Initialize variables
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op) #What change the value of theta
    best_theta = theta.eval()

Epoch 0 MSE = 33.66369
Epoch 100 MSE = 26.819557
Epoch 200 MSE = 26.31534
Epoch 300 MSE = 26.100836
Epoch 400 MSE = 25.950356
Epoch 500 MSE = 25.841347
Epoch 600 MSE = 25.762253
Epoch 700 MSE = 25.704832
Epoch 800 MSE = 25.663153
Epoch 900 MSE = 25.632887
